In [1]:
import random
import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Read CSV file
data = pd.read_excel('sim_eventlog.xlsx')

In [3]:
data

,CaseID,Activity_order,Activity,AssignedTimeStamp,StartTimeStamp,EndTimeStamp,Duration (hr)
0,C-1,1,Start,2023-01-01 09:00:00,NaN,NaN,0
1,C-1,2,Activity A,NaT,NaN,NaN,1
2,C-1,3,Activity B,NaT,NaN,NaN,2
3,C-1,4,Activity C,NaT,NaN,NaN,3
4,C-2,1,Start,2023-01-02 11:00:00,NaN,NaN,0
5,C-2,2,Activity A,NaT,NaN,NaN,1
6,C-2,3,Activity B,NaT,NaN,NaN,2
7,C-2,4,Activity C,NaT,NaN,NaN,3
8,C-3,1,Start,2023-01-03 10:30:00,NaN,NaN,0
9,C-3,2,Activity A,NaT,NaN,NaN,1


In [4]:
# Convert Datetime column to the right format
data["AssignedTimeStamp"] = pd.to_datetime(data['AssignedTimeStamp'])
data["StartTimeStamp"] = pd.to_datetime(data['StartTimeStamp'])
data["EndTimeStamp"] = pd.to_datetime(data['EndTimeStamp'])

# Convert duration column to Timedelta
data["Duration (hr)"] = pd.to_timedelta(data["Duration (hr)"] * 3600, unit='s')

In [5]:
data.head()

,CaseID,Activity_order,Activity,AssignedTimeStamp,StartTimeStamp,EndTimeStamp,Duration (hr)
0,C-1,1,Start,2023-01-01 09:00:00,NaT,NaT,0 days 00:00:00
1,C-1,2,Activity A,NaT,NaT,NaT,0 days 01:00:00
2,C-1,3,Activity B,NaT,NaT,NaT,0 days 02:00:00
3,C-1,4,Activity C,NaT,NaT,NaT,0 days 03:00:00
4,C-2,1,Start,2023-01-02 11:00:00,NaT,NaT,0 days 00:00:00


In [6]:
# Establish key variables
SimulationStartDate = "2023-01-02 09:00:00"  # Fixed typo in the initial date format
day_length = 8  # hours
daily_availability = 8  # hours - this means resource availability
waiting_delay = 5  # minutes of delay before starting any activity (representing queue)

In [7]:
# Convert variables to the right format
SimulationStartDate = datetime.strptime(SimulationStartDate, '%Y-%m-%d %H:%M:%S')
day_length = timedelta(hours=day_length)
daily_availability = timedelta(hours=daily_availability)
waiting_delay = timedelta(minutes=waiting_delay)

In [8]:
for i in range(10):
    StartDateTime = SimulationStartDate + timedelta(days=i)
    day_end = StartDateTime + day_length
    adjusted_daily_availability = daily_availability
    
    for j in range(20):
        # Capture first activity index, assigned date, and duration
        data = data.sort_values(by='AssignedTimeStamp')
        
        # Step: running the day if there are no more activities
        if not len(data[(data['AssignedTimeStamp'] <= day_end) & (data['StartTimeStamp'].isnull())]) > 0:
            break
        
        activity_index = data[(data['AssignedTimeStamp'] < day_end) & (data['StartTimeStamp'].isnull())].iloc[0].name
        case_assigned_date = data[(data['AssignedTimeStamp'] <= day_end) & (data['StartTimeStamp'].isnull())]['AssignedTimeStamp'].iloc[0]
        activity_duration = data[(data['AssignedTimeStamp'] <= day_end) & (data['StartTimeStamp'].isnull())]["Duration (hr)"].iloc[0]
        
        # Step: running the day if we ran out of daily availability
        if not adjusted_daily_availability - activity_duration >= timedelta(hours=0):
            break
        
        # Calculate start and end timestamps
        case_start_time = max(case_assigned_date, StartDateTime) + waiting_delay
        case_end_time = case_start_time + activity_duration
        
        # Reduce daily availability based on work done
        adjusted_daily_availability -= activity_duration
        
        # Record start and end timestamp for the current activity
        data.at[activity_index, 'StartTimeStamp'] = case_start_time
        data.at[activity_index, 'EndTimeStamp'] = case_end_time
        
        # Find the next step and populate the Assigned date
        CaseID = data.at[activity_index, 'CaseID']
        Activity_order = data.at[activity_index, 'Activity_order']
        
        # Skip the error if this was the last activity in the case
        try:
            next_activity_index = data[(data['CaseID'] == CaseID) & (data['Activity_order'] == Activity_order + 1)].index[0]
            data.at[next_activity_index, 'AssignedTimeStamp'] = case_end_time
        except IndexError:
            pass

In [9]:
data

,CaseID,Activity_order,Activity,AssignedTimeStamp,StartTimeStamp,EndTimeStamp,Duration (hr)
0,C-1,1,Start,2023-01-01 09:00:00,2023-01-02 09:05:00,2023-01-02 09:05:00,0 days 00:00:00
1,C-1,2,Activity A,2023-01-02 09:05:00,2023-01-02 09:10:00,2023-01-02 10:10:00,0 days 01:00:00
2,C-1,3,Activity B,2023-01-02 10:10:00,2023-01-02 10:15:00,2023-01-02 12:15:00,0 days 02:00:00
4,C-2,1,Start,2023-01-02 11:00:00,2023-01-02 11:05:00,2023-01-02 11:05:00,0 days 00:00:00
5,C-2,2,Activity A,2023-01-02 11:05:00,2023-01-02 11:10:00,2023-01-02 12:10:00,0 days 01:00:00
6,C-2,3,Activity B,2023-01-02 12:10:00,2023-01-02 12:15:00,2023-01-02 14:15:00,0 days 02:00:00
3,C-1,4,Activity C,2023-01-02 12:15:00,2023-01-03 09:05:00,2023-01-03 12:05:00,0 days 03:00:00
7,C-2,4,Activity C,2023-01-02 14:15:00,2023-01-03 09:05:00,2023-01-03 12:05:00,0 days 03:00:00
8,C-3,1,Start,2023-01-03 10:30:00,2023-01-03 10:35:00,2023-01-03 10:35:00,0 days 00:00:00
9,C-3,2,Activity A,2023-01-03 10:35:00,2023-01-03 10:40:00,2023-01-03 11:40:00,0 days 01:00:00
